# Using jupyter-jchannel in a Colab notebook

## Installing

You can use your favorite manager to install the [PyPI package](https://pypi.org/project/jupyter-jchannel/).

In [ ]:
!pip install jupyter-jchannel

In a remote notebook, you might need aditional resources to setup a tunnel.

For this specific example, we need to install the [Cloudflare Daemon](https://developers.cloudflare.com/cloudflare-one/connections/connect-networks/downloads/) to setup a [Cloudflare Tunnel](https://developers.cloudflare.com/cloudflare-one/connections/connect-networks/).

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

Also, to avoid polluting this notebook, we also install [a simple wrapper module](https://github.com/hashiprobr/jupyter-jchannel/blob/main/examples/cloudflare.py) to encapsulate this daemon.

In [ ]:
!wget https://raw.githubusercontent.com/hashiprobr/jupyter-jchannel/main/examples/cloudflare.py

## Starting a server

For basic usage, you only need to import the `jchannel` module.

In this specific example, we also need to import the aforementioned wrapper module.

In [ ]:
import jchannel
import cloudflare

A [jchannel Server](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server) instance runs alongside a Jupyter server instead of over it. Therefore, it needs its own local address.

In [ ]:
HOST = '127.0.0.1'
PORT = 8889

First, the `Tunnel` class of the wrapper module encapsulates the Cloudflare Daemon.

In [ ]:
tunnel = cloudflare.Tunnel('./cloudflared')

Then, the `ingress` method runs the tunnel and returns its URL.

In [ ]:
url = tunnel.ingress(HOST, PORT)

Finally, the asynchronous [start function](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.html#jchannel.start) instantiates a server, starts this server and returns it.

Notice that this function receives the tunnel URL as an argument.

In [ ]:
server = await jchannel.start(host=HOST, port=PORT, url=url)

## Opening a channel

A [server channel](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel) uses a server to call frontend JavaScript code from kernel Python code. The asynchronous [open method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server.open) instantiates a channel, opens this channel and returns it.

This method receives a string representing a JavaScript function. This function should receive a [client representation of the same channel](https://hashiprobr.github.io/jupyter-jchannel-client/Channel.html) and initialize it. The most important part of this initialization is setting the `handler` property to an object. The methods of this object define the API at the frontend.

The example below sets `handler` to an object that wraps the [padStart](https://developer.mozilla.org/docs/Web/JavaScript/Reference/Global_Objects/String/padStart) and [padEnd](https://developer.mozilla.org/docs/Web/JavaScript/Reference/Global_Objects/String/padEnd) string methods to provide indentation utilities.

In [ ]:
channel = await server.open('''
    (channel) => {
        channel.handler = {
            indentLeft(line, count) {
                return line.padStart(line.length + count);
            },
            indentRight(line, count) {
                return line.padEnd(line.length + count);
            },
        };
    }
''')

And that's it! You can now call any method of the API from the notebook.

## Calling the methods

The [call method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel.call) returns a [task](https://docs.python.org/3/library/asyncio-task.html#asyncio.Task) that can be awaited to retrieve the result whenever you want.

In [ ]:
task1 = channel.call('indentLeft', 'hello', 4)
task2 = channel.call('indentRight', 'world', 4)

In [ ]:
# some code you want to run before retrieving the results

In [ ]:
await task2

In [ ]:
await task1

In particular, awaiting immediately ensures synchronous execution, without the
need for sleeping and/or polling.

In [ ]:
await channel.call('indentLeft', 'hello', 4)

Furthermore, if the frontend throws a JavaScript exception, the task wraps it into a Python exception.

In [ ]:
await channel.call('indentRight', 4, 'world') # arguments in wrong order